In [100]:
using JuMP, DataFrames, Gurobi, FileIO, TextParse, CSVFiles

# Set-working directory
#DIR = "C:\\Users\\will-\\GoogleDrive\\UCBerkeley\\Research\\Papers\\2018 Off-grid\\"
#DIR = "C:\\Users\\Will\\GoogleDrive\\UCBerkeley\\Research\\Papers\\2018 Off-grid\\Analysis\\"
DIR = "G:\\Team Drives\\grid_defect_data\\"
OUT = "Analysis\\out"
INPUT = "Analysis\\in"

##### CREATE MODEL RUN ######
# Set constants
LOAD_SHED = [0.25, 0.0001]
BAT_COST = 500 # $/kWh
PV_COST = 3000 # $/kW
BAT_EFF = 0.92
#annual rates
int_rate = 0.03 # percentage interest rate
bat_life = 20 # years
sol_life = 25 # years
BAT_RATE = int_rate / (1 - (1+int_rate)^(-bat_life))
PV_RATE = int_rate / (1 - (1+int_rate)^(-sol_life));


In [12]:
# identify geography IDs to loop through
ID_G = load(DIR * INPUT * "\\optimization_list.csv") |> DataFrame

,,row_in_2,county,state,lat.x,lon,timezone,elevation,tz,ID,lat.y,long
1,3152,901,allen,ky,36.731,-86.1466,-6,244,US/Central,746716,36.983,-86.433
2,3153,901,barren,ky,36.9383,-85.9236,-6,218,US/Central,746716,36.983,-86.433
3,3141,900,benton,tn,36.0874,-88.0521,-6,154,US/Central,746710,36.667,-87.483


In [101]:
script = "C:\\Users\\wgorman\\Desktop\\grid_defection\\"
include(script * "optimization_model.jl")

solar_opt (generic function with 1 method)

In [102]:
#Run Model
timing = @elapsed for i in 1:2
    
    result, outcome = solar_opt(ID_G, LOAD_SHED, BAT_COST, BAT_RATE, BAT_EFF, PV_COST, PV_RATE, DIR, INPUT, i) 
    
    if (i == 1)
        global results = result
    else
        append!(results, result)
    end
    
    if (i == 1)
        global outcomes = outcome
    else
        append!(outcomes, outcome)
    end

    print(i)
end

Academic license - for non-commercial use only
Optimize a model with 473044 rows, 394205 columns and 1298736 nonzeros
Coefficient statistics:
  Matrix range     [5e-06, 1e+00]
  Objective range  [4e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 3e+04]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 78847 rows and 78846 columns
Presolve time: 1.33s
Presolved: 394197 rows, 315359 columns, 1141041 nonzeros

Ordering time: 0.09s

Barrier statistics:
 Dense cols : 2
 AA' NZ     : 9.834e+05
 Factor NZ  : 4.712e+06 (roughly 300 MBytes of memory)
 Factor Ops : 7.063e+07 (less than 1 second per iteration)
 Threads    : 3

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.28384218e+05  0.00000000e+00  3.12e+07 0.00e+00  1.53e+02     2s
   1   2.88301231e+05  4.34641681e+03  7.91e+06 1.67e-01  5.91e+01     2s
   2   3.39376726e+05  1.89169

109.103231625

In [104]:
save(DIR * OUT * "\\outcomes.csv", outcomes)

In [55]:
save(DIR * OUT * "\\results.csv", results)